In [ ]:
from __future__ import print_function
import socket
import json
import sys
import threading

from satori.rtm.client import make_client, SubscriptionMode

channel = "US-Bike-Sharing-Channel"
endpoint = "wss://open-data.api.satori.com"
appkey = "650Ce77b5f6f888F14addb8919D8D264"

stream_host = 'localhost'
stream_port = 9982

def stream_data():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((stream_host, stream_port))
    print('Waiting for Listener...')
    s.listen(10)
    conn, addr = s.accept()

    with make_client(
        endpoint=endpoint, appkey=appkey) as client:
        print('Connected!')

        mailbox = []
        got_message_event = threading.Event()

        class SubscriptionObserver(object):
            def on_subscription_data(self, data):
                for message in data['messages']:
                    mailbox.append(message)
                    got_message_event.set()

        subscription_observer = SubscriptionObserver()
        client.subscribe(
            channel,
            SubscriptionMode.SIMPLE,
            subscription_observer)

        if not got_message_event.wait(10):
            print("Timeout while waiting for a message")
            sys.exit(1)

        try:
            while True:
                for message in mailbox:
                    # print(message)
                    msg_json = json.dumps(message, ensure_ascii=False)
                    msg_end = msg_json + '\n'
                    conn.send(msg_end.encode('utf-8'))
                    #conn.send(msg_json.encode('utf-8') + '\n')
        except Exception as e:
            print(str(e))
        finally:
            conn.close()

In [ ]:
stream_data()